In [1]:
import os
import math
import numpy as np
import pandas as pd

In [3]:
import torch
import torchvision

from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.transforms import Resize, ToTensor, Normalize, Compose

# Dataset 불러오기
Dataset.py에서 정의한 MaskDataset을 불러옵니다.

In [23]:
from dataset import MaskBaseDataset, TestDataset

In [7]:
TRAIN_DATA_PATH = "../input/data/train/images"
TEST_DATA_PATH = "../input/data/eval"

In [8]:
BATCH_SIZE = 128
DROP_LAST = False

#### transform 정의
1. torchvision에서 제공하는 다양한 함수를 사용하여 입력으로 주어지는 이미지를 전처리합니다.
2. 512 × 384의 크기로 영상을 Resize하고, 데이터를 Tensor로 바꿉니다.
3. 0~255 사이의 값을 지닌 RGB 값을 각각 0과 1의 범위로 정규화합니다.

In [9]:
transform = Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

#### Train Dataset과 Validation Dataset 나누기
Train Dataset과 Validation Dataset을 random으로 8:2 비율로 구분합니다.

In [10]:
maskDataset = MaskBaseDataset(data_dir=TRAIN_DATA_PATH)
maskDataset.set_transform(transform)
train_dataset, val_dataset = maskDataset.split_dataset()

print(f'Shape of Dataset')
print(f'Train Dataset: {len(train_dataset)}')
print(f'Validation Dataset: {len(val_dataset)}')

Shape of Dataset
Train Dataset: 15120
Validation Dataset: 3780


In [11]:
train_dataloader = DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    sampler=None, collate_fn=None, drop_last=DROP_LAST
)

val_dataloader = DataLoader(
    dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True,
    sampler=None, collate_fn=None, drop_last=DROP_LAST
)

In [12]:
next(iter(train_dataloader))[0].shape

torch.Size([128, 3, 512, 384])

# Model 정의
하나는 Pretrained 모델을, 다른 하나는 Customized 모델을 사용하여 두 모델의 결과를 확인해 봅시다.

In [13]:
NUM_CLASS = 18
NUM_EPOCH = 5
LEARNING_RATE = 1e-4

In [14]:
resnet18 = torchvision.models.resnet18(pretrained=True)

In [15]:
resnet18.fc = torch.nn.Linear(in_features= 512, out_features=NUM_CLASS, bias=True)

torch.nn.init.kaiming_normal_(resnet18.fc.weight)
stdv = 1. / math.sqrt(resnet18.fc.weight.size(1))
resnet18.fc.bias.data.uniform_(-stdv, stdv)
resnet18.fc.weight.shape[0]

18

In [16]:
# M1 Mac은 CUDA를 지원하지 않습니다. 😭
# device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
device

device(type='cuda', index=0)

In [17]:
resnet18.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train": train_dataloader,
    "validation": val_dataloader
}

In [18]:
best_val_accuracy = 0
best_test_loss = np.inf

for epoch in range(NUM_EPOCH):
    print('*** Epoch {} ***'.format(epoch))
    for phase in ["train", "validation"]:
        running_loss, running_acc = 0.0, 0.0

        if phase == "train":
            resnet18.train()
        else:
            resnet18.eval()

        index = 0
        for index, (inputs, labels) in enumerate(tqdm(dataloaders[phase])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == "train"):
                outputs = resnet18(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == "train":
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.shape[0]
            running_acc += torch.sum(preds == labels.data)

        if DROP_LAST:
            num_input = (index + 1) * BATCH_SIZE
        else:
            num_input = len(dataloaders[phase].dataset)
        running_loss = running_loss / num_input
        running_acc = running_acc / num_input

        print('{} Loss: {:.4f} / Acc: {:.4f}'.format(phase, running_loss, running_acc))

        if phase == "validation":
            if (best_test_loss > running_loss):
                best_test_loss = running_loss
            if (best_val_accuracy < running_acc):
                best_val_accuracy = running_acc

print("Training Result")
print(f'Least Loss: {best_test_loss:.4f} / Best Accuracy: {best_val_accuracy:.4f}')

  0%|          | 0/119 [00:00<?, ?it/s]

*** Epoch 0 ***


  0%|          | 0/30 [00:00<?, ?it/s]

train Loss: 0.6387 / Acc: 0.8195


  0%|          | 0/119 [00:00<?, ?it/s]

...

validation Loss: 0.0590 / Acc: 0.9817
*** Epoch 5 ***


  0%|          | 0/30 [00:00<?, ?it/s]

train Loss: 0.0032 / Acc: 0.9999


100%|██████████| 30/30 [00:26<00:00,  1.14it/s]

validation Loss: 0.0523 / Acc: 0.9831
Training Result
Least Loss: 0.0523 / Best Accuracy: 0.9831


In [ ]:
submission = pd.read_csv(os.path.join(TEST_DATA_PATH, 'info.csv'))
image_dir = os.path.join(TEST_DATA_PATH, 'images')

image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

device = torch.device('cuda')
model = resnet18.to(device)
model.eval()

all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

submission.to_csv(os.path.join(TEST_DATA_PATH, 'submission.csv'), index=False)
print('Inference Completed')